In [14]:
import numpy as np
import scipy as sp
from scipy import io
import cv2 as cv 
import matplotlib.pyplot as plt
import pylab
import os # for reading all files in a folder
pylab.rcParams['figure.figsize'] = (12.0, 10.0)

# Condensation 7c

You'll have to call the code below four times (once each with 'll','lr', 'ul', 'ur': lower left, lower right, upper left, ipper right). For each time you'll get an estimated track (2D locations over time) of one of the corners of the pattern moved. Note that we're saving off the single MAP estimate at each frame, but the particles in the Particle Filter are attempting to keep track of the whole posterior distribution. 

Most TO DO parts below should be copy-paste from Practical9a and Practical9b.

computeLikelihood has been modified from its previous versions to cope with these particular gray-scale images.


Qustions you could ask yourself (not a TO DO): 
- Can you find a pattern between the movement of the particles and the movement of the camera relative to the scene?
- When is the MAP estimate especially good/bad?
- What is the relationship to the amount of noise? (--> experiment with different levels!)
- Why do different numbers of particles lead to very differenet results (for example 50 vs. 2000)? 

In [15]:
#Likelihood function is simple patch similarity

def computeLikelihood(image, template):
    #opencv's available methods - experiment with these
    #careful what range the output is!
    methods = [cv.TM_CCOEFF, cv.TM_CCOEFF_NORMED, cv.TM_CCORR,
            cv.TM_CCORR_NORMED, cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]
    
    likelihood = cv.matchTemplate(image[:,:,2], template, methods[0])
    # (You can also try converting the image to greyscale instead of using the third channel as above with 
    # cv.cvtColor(image, cv.COLOR_BGR2GRAY))
    
    #we can pad to make this the size of the input image (for easier indexing)   
    pad_first = int(template.shape[0])
    pad_second = int(template.shape[1])
    pad_amounts = ((0, pad_first-1), (0, pad_second-1))
    likelihood = np.pad(likelihood, pad_amounts, 'constant')
    likelihood[likelihood<0] = 0 # to avoid negative weights 
    
    # apply a 10x10 averaging filter for stability. You can experiment with different sizes. 
    kernel = np.ones((10,10),np.float32)/100
    smoothed = cv.filter2D(likelihood,-1,kernel) 
    return smoothed 

In [16]:

def HW2_Practical7c(corner):
    template = sp.io.loadmat(corner+'.mat')['pixelsTemplate']
    #let's show the template
    # print('We are matching this template with shape: ', template.shape)
    # plt.imshow(template)
    # plt.show()

    # Load all images in folder
    images = []
    iFrame = 0
    folder = 'Pattern01/'
    lst = os.listdir(folder)
    lst.sort()

    for frameNum in lst:
        images.append(cv.imread(folder+frameNum))
        iFrame += 1
    # plot first image 
    # plt.imshow(images[0])
    # plt.show()

    imgHeight, imgWidth, colors = images[0].shape
    numParticles = 2000;
    weight_of_samples = np.ones((numParticles,1))

    # TO DO: normalize the weights (may be trivial this time) [done]
    weight_of_samples = weight_of_samples / np.sum(weight_of_samples)

    # Initialize which samples from "last time" we want to propagate: all of
    # them!:
    samples_to_propagate = range(0, numParticles)


    # ============================
    # NOT A TO DO: You don't need to change the code below, but eventually you may
    # want to vary the number of Dims (compare for example to lab 9b) 
    numDims_w = 4; 
    # Here we randomly initialize some particles throughout the space of w:
    particles_old = np.random.rand(numParticles, numDims_w)
    particles_old[:,0] = particles_old[:,0] * imgHeight
    particles_old[:,1] = particles_old[:,1] * imgWidth
    particles_old[:,2] = 10.0 * np.random.randn(numParticles)
    particles_old[:,3] = 10.0 * np.random.randn(numParticles)
    # ============================

    #Initialize a temporary array r to store the per-frame MAP estimate of w. This is what we'll return in the end.
    #r = np.zeros((iFrame, numDims_w));
    #感觉这里直接抛弃后两维没关系，后面的只是用来预测，这里的r代表return，return的值是用来标记图上点的位置地
    r = np.zeros((iFrame, 2));

    for iTime in range(iFrame):
        print('Processing Frame', iTime)
        # TO DO: compute the cumulative sume of the weights. [done]
        cum_hist_of_weights = np.cumsum(weight_of_samples)
        #print(weight_of_samples)


        # ==============================================================
        # Resample the old distribution at time t-1, and select samples, favoring
        # those that had a higher posterior probability.
        # ==============================================================
        samples_to_propagate = np.zeros(numParticles, dtype=np.int32)

        # Pick random thresholds in the cumulative probability's range [0,1]:
        some_threshes = np.random.rand(numParticles)


        # For each random threshold, find which sample in the ordered set is
        # the first one to push the cumulative probability above that
        # threshold, e.g. if the cumulative histogram goes from 0.23 to 0.26
        # between the 17th and 18th samples in the old distribution, and the
        # threshold is 0.234, then we'll want to propagate the 18th sample's w
        # (i.e. particle #18).

        for sampNum in range(numParticles): 
            thresh = some_threshes[sampNum]
            for index in range (numParticles):
                if cum_hist_of_weights[index] > thresh:
                    break
            samples_to_propagate[sampNum] = index

        # Note: it's ok if some of the old particles get picked repeatedly, while
        # others don't get picked at all.

        # Predict where the particles we sampled from the old distribution of 
        # state-space will go in the next time-step. This means we have to apply 
        # the motion model to each old sample.
        particles_new = np.zeros_like(particles_old)
        for particleNum in range(numParticles):
            # TO DO: Incorporate some noise, e.g. Gaussian noise with std 20,
            # into the current location (particles_old), to give a Brownian
            # motion model.
            noisePosition_std = 10
            noiseVelocity_std =  5
            # Adding Gaussian noise to the position and velocity
            noisePosition = np.random.normal(0, noisePosition_std, 2)
            noiseVelocity = np.random.normal(0, noiseVelocity_std, 2)
            
            selected_particle_index = samples_to_propagate[particleNum]

            # Updating the position and velocity of the selected particle
            particles_new[particleNum, :2] = particles_old[selected_particle_index, :2] + noisePosition
            particles_new[particleNum, 2:] = particles_old[selected_particle_index, 2:] + noiseVelocity

            #particles_new[particleNum, :] = np.round(particles_new[particleNum, :]) # Round the particles_new to simplify Likelihood evaluation.
            
        # TO DO: Not initially, but change the motion model above to have
        # different degrees of freedom, and optionally completely different
        # motion models. See Extra Credit for more instructions.

        #calculate likelihood function
        likelihood = computeLikelihood(images[iTime], template)

        #plot results
        # f, axarr = plt.subplots(1, 2)
        # axarr[0].imshow(images[iTime])
        # axarr[0].set_title('Particles')
        # # now draw the particles onto the image
        # axarr[0].plot(particles_new[:,1]+template.shape[1]/2, particles_new[:,0]+template.shape[0]/2, 'rx')

        # #plot the likelihood
        # axarr[1].imshow(likelihood)
        # axarr[1].set_title('Likelihood')

        # From here we incorporate the data for the new state (time t):
        # The new particles accompanying predicted locations in state-space
        # for time t, are missing their weights: how well does each particle
        # explain the observations x_t?
        for particleNum in range(numParticles):

            # Convert the particle from state-space w to measurement-space x:
            # Note: that step is trivial here since both are in 2D space of image
            # coordinates

            # Within the loop, we evaluate the likelihood of each particle:
            particle = particles_new[particleNum, :]
            # Check that the predicted location is a place we can really evaluate
            # the likelihood.
            inFrame = particle[0] >= 0.0 and  particle[0] <= imgHeight and particle[1] >= 0.0 and particle[1] <= imgWidth
            if inFrame:
                minX = particle[1]
                minY = particle[0]

                weight_of_samples[particleNum] = likelihood[int(minY), int(minX)]

            else:
                weight_of_samples[particleNum] = 0.0

        # TO DO: normalize the weights [done]
        weight_of_samples /= np.sum(weight_of_samples)
        
        # find the location of the particle with highest weight
        indices = np.argsort(weight_of_samples,0)
        bestScoringParticles = particles_new[np.squeeze(indices[-15:]), :]
        # plt.plot(bestScoringParticles[-1:,1], bestScoringParticles[-1:,0], 'rx')
        # Return the MAP of middle position. Add template.shape/2 because matchTemplate finds the position of the upper left corner 
        # of the template. We want to plot the centre of the template. 
        r[iTime,:] = bestScoringParticles[-1,1]+template.shape[1]/2,bestScoringParticles[-1,0]+template.shape[0]/2
        # print(r[iTime,:])   
        # plt.show()

        # #print the original image and the position of the tracked corner.
        # plt.imshow(images[iTime])
        # plt.plot(r[iTime,0],r[iTime,1],'rx')
        # plt.show()
        # # Now we're done updating the state for time t. 
        # For Condensation, just clean up and prepare for the next round of 
        # predictions and measurements:
        particles_old = particles_new

    return r

In [17]:
#r = HW2_Practical7c('ul')

In [18]:
def save_results(data, file_name, folder='results'):
    # 检查并创建文件夹
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # 完整的文件路径
    file_path = os.path.join(folder, file_name)
    
    # 写入数据到文件
    with open(file_path, 'w') as file:
        for item in data:
            file.write(str(item) + '\n')


LLs = HW2_Practical7c('ll')
save_results(LLs, 'LLs_results.txt')

LRs = HW2_Practical7c('lr')
save_results(LRs, 'LRs_results.txt')

ULs = HW2_Practical7c('ul')
save_results(ULs, 'ULs_results.txt')

URs = HW2_Practical7c('ur')
save_results(URs, 'URs_results.txt')

Processing Frame 0
Processing Frame 1
Processing Frame 2
Processing Frame 3
Processing Frame 4
Processing Frame 5
Processing Frame 6
Processing Frame 7
Processing Frame 8
Processing Frame 9
Processing Frame 10
Processing Frame 11
Processing Frame 12
Processing Frame 13
Processing Frame 14
Processing Frame 15
Processing Frame 16
Processing Frame 17
Processing Frame 18
Processing Frame 19
Processing Frame 20
Processing Frame 21
Processing Frame 22
Processing Frame 23
Processing Frame 24
Processing Frame 25
Processing Frame 26
Processing Frame 27
Processing Frame 28
Processing Frame 29
Processing Frame 30
Processing Frame 31
Processing Frame 32
Processing Frame 33
Processing Frame 34
Processing Frame 35
Processing Frame 36
Processing Frame 37
Processing Frame 38
Processing Frame 39
Processing Frame 40
Processing Frame 41
Processing Frame 42
Processing Frame 43
Processing Frame 44
Processing Frame 45
Processing Frame 46
Processing Frame 47
Processing Frame 48
Processing Frame 49
Processing